In [ ]:
#!pip install azure-cosmos

In [ ]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions
import uuid

# Configuration
ENDPOINT = "https://pvn-cosmos.documents.azure.com:443/"
KEY = "tpKr6DFYfkTbrmNvH7SLWxZ7xDRqKD4wNdINoFZjGnovSlV6WunxPt4bQGsr2GgDb6vmUDDq8nKvACDbfy64KQ==;"
DATABASE_NAME = "Blobdb"
CONTAINER_NAME = "Users"

In [ ]:
# Initialize client
client = CosmosClient(ENDPOINT, KEY)
database = client.get_database_client(DATABASE_NAME)
container = database.get_container_client(CONTAINER_NAME)

In [ ]:
items = list(container.query_items(
    query = "SELECT c.name from c where c.userid = 'U2'", 
    enable_cross_partition_query=True
))

In [ ]:
print(items)

In [ ]:
container_name = "product"
partition_key_path = "/productid"

In [15]:
from azure.cosmos import PartitionKey

container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path=partition_key_path),
    offer_throughput=400)

In [ ]:
# List all containers
print("Containers in database:")
for container in database.list_containers():
    print("-", container['id'])

In [ ]:
import uuid

# Sample product item
product_item = {
    "id": str(uuid.uuid4()),      # unique document id
    "productid": "P1001",         # partition key value
    "name": "Laptop",
    "price": 75000,
    "category": "Electronics"
}

container.create_item(body=product_item)
print("Inserted item:", product_item)


In [ ]:
# Query all items
print("\nReading items in container:")
for item in container.read_all_items():
    print(item)

# OR Query using SQL syntax
query = "SELECT * FROM c WHERE c.category='Electronics'"
items = list(container.query_items(query=query, enable_cross_partition_query=True))
print("\nQueried items:", items)


In [ ]:
# Let's pick an item by productid
query = "SELECT * FROM c WHERE c.productid='P1001'"
items = list(container.query_items(query=query, enable_cross_partition_query=True))
if items:
    item = items[0]
    print("Before update:", item)
    
    # Modify the item
    item["price"] = 78000
    item["stock"] = 25

    # Replace the item
    container.replace_item(item=item['id'], body=item)
    print("Updated item:", item)


In [ ]:
item_id = item["id"]
partition_key = item["productid"]

container.delete_item(item=item_id, partition_key=partition_key)
print(f"Deleted item with id: {item_id}")


In [ ]:
remaining_items = list(container.read_all_items())
print("\nRemaining items after deletion:")
for it in remaining_items:
    print(it)